In [1]:
import xlrd 
#import glob
import os
import pandas as pd
import math


inputDIR = r'P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final'
extens = '.xlsx'
outputDIR = r'P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final'


fuString = 'FU'
catchString = 'Catchment'

dictOfDataFrames = {}
dataFrameCols = [catchString, fuString, 'Constituent', 'Parameter', 'Value']
#df = pd.DataFrame(columns=dataFrameCols)

#allConsNames = ['Fine_SED', 'Coarse_SED', 'N_DIN', 'N_DON', 'PN', 'P_FRP', 'P_DOP', 'PP']
#Intentionally gone with upper case for matching
allConsDict = {'FINE_SED':'Sediment - Fine', 'COARSE_SED':'Sediment - Coarse', 'N_DIN':'N_DIN', 'N DIN':'N_DIN',
               'N_DON':'N_DON', 'N DON':'N_DON', 'N_PARTICULATE':'N_Particulate', 'PN':'N_Particulate', 'N_PART':'N_Particulate',
               'N PART':'N_Particulate', 'P_FRP':'P_FRP', 'P FRP':'P_FRP', 'P_DOP':'P_DOP', 'P DOP':'P_DOP',
               'P_PARTICULATE':'P_Particulate', 'PP':'P_Particulate', 'P_PART':'P Particulate', 'P PART':'P_Particulate'}

allFlowsDict = {'QF':['EMC','START'], 'SF':['DWC','START'], ' QFKG':['EMC','END'], ' SFKG':['DWC','END'],
                '_QFKG':['EMC','END'], '_SFKG':['DWC','END']}

#Jesus... naming of sheets not consistent
allRegions = {'BM':'.', 'BU':' ', 'CY':' ', 'FI':'.', 'MW':'.', 'WT':' '}
#allRegions = ['BM']

#Intentionally gone with upper case for matching
#allScenarios = {'BASE':'BASE', 'DTOC':'DtoC', 'ALLB':'ALLB', 'ALLA':'ALLA'}
allScenarios = ['BASE', 'DTOC', 'ALLB', 'ALLA']
#allScenarios = ['BASE']

fileNamerequisite = 'LMs_by_Catchment'

regressionValIdent = 'TI0'
transformVal = 0.2
slopeColOffSet = 2






In [2]:
#list files
relevantFiles = []
for file in os.listdir(inputDIR):
    if file.endswith(extens):
        #print(file)
        theBits = file.split('_')
        
        if(theBits[0] in allRegions.keys()):
            if(theBits[1].upper() in allScenarios):
                relevantFiles.append(file)

#print(relevantFiles)
print("All Done")

All Done


In [3]:
# To open Workbook
dictOfDataFrames = {}
for relFile in relevantFiles:
    theBits = relFile.split('_')
    theReg = theBits[0]
    
    print('This region is ' + theReg)
    
    if not theReg in dictOfDataFrames.keys():
        print('Adding this region to dictionary ' + theReg)
        dictOfDataFrames[theReg] = {}
    
    sheetNamesUpperWithIdx = {}
    wb = xlrd.open_workbook(os.path.join(inputDIR, relFile))
    print(relFile + " has this many sheets: " + str(wb.nsheets))
    theIdx = 0
    for sName in wb.sheet_names():
        #print(sName.upper())
        sheetNamesUpperWithIdx[sName.upper()] = theIdx
        theIdx += 1
    
    for scen in allScenarios:
        
        curRecord = -99
        if not scen in dictOfDataFrames[theReg].keys():
            print('Adding this scenario to dictionary ' + scen)
            dictOfDataFrames[theReg][scen] = pd.DataFrame(columns=dataFrameCols)
            curRecord = 0
        
        
        for flowType in allFlowsDict.keys():
            #First entry in list tells if EMC/DWC
            theParam = allFlowsDict[flowType][0]
            
            for cons in allConsDict.keys():
                theCatchModelCons = allConsDict[cons]
                
                #BASE. or BASEspace
                firstNameBit = scen + allRegions[theReg]
                
                expectedName = firstNameBit
                
                if(allFlowsDict[flowType][1] == 'START'):
                    expectedName = expectedName + flowType + "_" + cons
                    if(allRegions[theReg] == '.'):
                        #BM, FI and MW use a dot, and also include ' ALL'
                        expectedName = expectedName + ' ALL'
                
                else:
                    #Stuff goes at the end...
                    expectedName = expectedName + cons + flowType
                
                #print('Looking for sheet ' + expectedName)
                sheet = None
                if(expectedName in sheetNamesUpperWithIdx.keys()):
                    #Righto....
                    print('Accessing sheet ' + expectedName + ' at Idx: ' + str(sheetNamesUpperWithIdx[expectedName]))
                    sheet = wb.sheet_by_index(sheetNamesUpperWithIdx[expectedName])
                    print('Sheet ' + expectedName + ' with Nrows: ' + str(sheet.nrows) + ' and Ncols: ' + str(sheet.ncols))
                    #sheet = wb.sheet_by_index(expectedName)
                    rowIdx = 1000000

                    #cell_value(row, col)
                    for i in range(sheet.nrows):
                        #print("Checking row " + str(i))
                        #print("Val: " + str(sheet.cell_value(i, 1)))
                        if sheet.cell_value(i, 1) == fuString:
                            rowIdx = i + 1
                            break
                    
                    lastGoodColIdx = -99
                    for j in range(sheet.nrows):
                        if(j >= rowIdx):
                            
                            theFU = sheet.cell_value(j, 1)
                            #theCat = 'SC #' + str(sheet.cell_value(j, 2))
                            rawCatVal = sheet.cell_value(j, 2)
                            theCat = ''
                            
                            if (isinstance(rawCatVal, str)):
                                theCat = 'SC #' + rawCatVal
                            else:
                                theCat = 'SC #{:.0f}'.format(sheet.cell_value(j, 2),0) 
                            
                            reqColIdx = -99
                            
                            for k in range(sheet.ncols):
                                if(sheet.cell_value(j, k) == regressionValIdent):
                                    reqColIdx = k + slopeColOffSet
                                    break
                            
                            if(reqColIdx < sheet.ncols and reqColIdx >= 0):
                                lastGoodColIdx = reqColIdx       
                            
                            if(reqColIdx < 0):
                                print("Got lower reqCol for row : " + str(j) + " needing col: " + str(reqColIdx))
                                #Maybe there was a missing cell (does happen)
                                #grab old val
                                reqColIdx = lastGoodColIdx
                                
                            theRawVal = sheet.cell_value(j, reqColIdx)
                            theTransVal = 0
                            
                            if theRawVal == '#N/A':
                                print('Had to handle error at Idx: ' + str(j))
                                theRawVal = 0
                            
                            if theRawVal > 0:
                                theTransVal = math.pow(theRawVal, 1/transformVal) * 1000
                            
                            dictOfDataFrames[theReg][scen].loc[curRecord] = [theCat, theFU, theCatchModelCons, theParam, theTransVal]
                            curRecord += 1

#print(sheetNamesUpperWithIdx)    
print("Done")

This region is BM
Adding this region to dictionary BM
BM_ALLA_LMs_by_Catchment_20200206_all_stats1.xlsx has this many sheets: 80
Adding this scenario to dictionary BASE
Adding this scenario to dictionary DTOC
Adding this scenario to dictionary ALLB
Adding this scenario to dictionary ALLA
Accessing sheet ALLA.QF_FINE_SED ALL at Idx: 0
Sheet ALLA.QF_FINE_SED ALL with Nrows: 2660 and Ncols: 60
Accessing sheet ALLA.QF_COARSE_SED ALL at Idx: 2
Sheet ALLA.QF_COARSE_SED ALL with Nrows: 2252 and Ncols: 60
Accessing sheet ALLA.QF_N_DIN ALL at Idx: 4
Sheet ALLA.QF_N_DIN ALL with Nrows: 113 and Ncols: 60
Accessing sheet ALLA.QF_PN ALL at Idx: 8
Sheet ALLA.QF_PN ALL with Nrows: 3423 and Ncols: 60
Got lower reqCol for row : 339 needing col: -99
Accessing sheet ALLA.QF_P_FRP ALL at Idx: 10
Sheet ALLA.QF_P_FRP ALL with Nrows: 113 and Ncols: 60
Accessing sheet ALLA.QF_P_DOP ALL at Idx: 12
Sheet ALLA.QF_P_DOP ALL with Nrows: 113 and Ncols: 60
Accessing sheet ALLA.QF_PP ALL at Idx: 14
Sheet ALLA.QF_PP A

Accessing sheet DTOC N_DIN_QFKG at Idx: 4
Sheet DTOC N_DIN_QFKG with Nrows: 3899 and Ncols: 60
Accessing sheet DTOC N_DON_QFKG at Idx: 8
Sheet DTOC N_DON_QFKG with Nrows: 3844 and Ncols: 60
Accessing sheet DTOC N_PARTICULATE_QFKG at Idx: 12
Sheet DTOC N_PARTICULATE_QFKG with Nrows: 4529 and Ncols: 60
Accessing sheet DTOC P_FRP_QFKG at Idx: 14
Sheet DTOC P_FRP_QFKG with Nrows: 4044 and Ncols: 60
Accessing sheet DTOC P_DOP_QFKG at Idx: 18
Sheet DTOC P_DOP_QFKG with Nrows: 4044 and Ncols: 60
Accessing sheet DTOC P_PARTICULATE_QFKG at Idx: 22
Sheet DTOC P_PARTICULATE_QFKG with Nrows: 4529 and Ncols: 60
Accessing sheet DTOC N_DIN_SFKG at Idx: 6
Sheet DTOC N_DIN_SFKG with Nrows: 3899 and Ncols: 60
Accessing sheet DTOC N_DON_SFKG at Idx: 10
Sheet DTOC N_DON_SFKG with Nrows: 3844 and Ncols: 60
Accessing sheet DTOC P_FRP_SFKG at Idx: 16
Sheet DTOC P_FRP_SFKG with Nrows: 3844 and Ncols: 60
Accessing sheet DTOC P_DOP_SFKG at Idx: 20
Sheet DTOC P_DOP_SFKG with Nrows: 3844 and Ncols: 60
This region

Accessing sheet DTOC.QF_COARSE_SED ALL at Idx: 2
Sheet DTOC.QF_COARSE_SED ALL with Nrows: 172 and Ncols: 60
Accessing sheet DTOC.QF_PN ALL at Idx: 8
Sheet DTOC.QF_PN ALL with Nrows: 1180 and Ncols: 60
Accessing sheet DTOC.QF_PP ALL at Idx: 14
Sheet DTOC.QF_PP ALL with Nrows: 1180 and Ncols: 60
This region is WT
Adding this region to dictionary WT
WT_ALLA_LMs_by_Catchment_20200129_all_processed_stats.xlsx has this many sheets: 110
Adding this scenario to dictionary BASE
Adding this scenario to dictionary DTOC
Adding this scenario to dictionary ALLB
Adding this scenario to dictionary ALLA
Accessing sheet ALLA QF_FINE_SED at Idx: 0
Sheet ALLA QF_FINE_SED with Nrows: 2533 and Ncols: 60
Accessing sheet ALLA QF_COARSE_SED at Idx: 4
Sheet ALLA QF_COARSE_SED with Nrows: 1711 and Ncols: 60
Accessing sheet ALLA SF_FINE_SED at Idx: 2
Sheet ALLA SF_FINE_SED with Nrows: 800 and Ncols: 60
Accessing sheet ALLA N DIN QFKG at Idx: 6
Sheet ALLA N DIN QFKG with Nrows: 2533 and Ncols: 60
Accessing sheet A

In [4]:
#dictOfDataFrames['BM']['BASE'].head(20)
##print(dictOfDataFrames)

In [5]:
#save output
for theReg in dictOfDataFrames.keys():
    for theScen in dictOfDataFrames[theReg]:
        #filtered.to_csv(outputFile, index=False)
        fName = theReg + "_" + theScen + "_EMCDWCVals_FromFreeman.csv"
        outputFile = os.path.join(inputDIR, fName)
        print("Saving " + outputFile)
        dictOfDataFrames[theReg][theScen].to_csv(outputFile, index=False)

print("All Done!")

Saving P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final\BM_BASE_EMCDWCVals_FromFreeman.csv
Saving P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final\BM_DTOC_EMCDWCVals_FromFreeman.csv
Saving P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final\BM_ALLB_EMCDWCVals_FromFreeman.csv
Saving P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final\BM_ALLA_EMCDWCVals_FromFreeman.csv
Saving P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final\BU_BASE_EMCDWCVals_FromFreeman.csv
Saving P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final\BU_DTOC_EMCDWCVals_FromFreeman.csv
Saving P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final\BU_ALLB_EMCDWCVals_FromFreeman.csv
Saving P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final\BU_ALLA_EMCDWCVals_FromFreeman.csv
Saving P:\projects\QWMN\ModelImprovementTWG\FreemanCook\DES_SOURCE_Final\CY_BASE_EMCDWCVals_FromFreeman.csv
Saving P:\projects\QWMN\Mode